In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns

### *** Next sections go over data preprocessing, whose results fed into the R file 'SL3-LendingClub_Project.R'  
### *** where survival analysis is done and the curves plotted.  Explanation of methodology given in R file.
### *** You can either go down line by line for every preprocessing step in order, or follow heading instructions to skip over intermediate steps to go directly to sections of interest

In [ ]:
# Read in dataframe
df_accepted1_sm_inversetransformed = pd.read_csv('https://www.dropbox.com/s/f9z2zmfsjnht0dl/df_accepted1_sm_inversetransformed.csv?dl=1')

### To go directly to dataframe with columns dropped and null values cleaned, read file below and go to cell 28

In [ ]:
### Dropped columns and dropped NAs dataframe
df_accepted4_sm_inversetransformed = pd.read_csv('https://www.dropbox.com/s/hl1fp2b50a2due8/df_accepted4_sm_inversetransformed.csv?dl=1')

### To go directly to filtered 36month term datasets, read file below and go to cell 38 (in this section continuous binned into categorical levels, 'profit & loss' column created, other cleaning steps done)

In [ ]:
df_accepted4a_36m_no_0_cox = pd.read_csv('https://www.dropbox.com/s/m8pnpz5anojf3tb/df_accepted4a_36m_no_0_cox.csv?dl=1')

### To go directly to filtered 60month term datasets, read file below and go to cell 43 (in this section continuous binned into categorical levels, 'profit & loss' column created, other cleaning steps done)

In [ ]:
df_accepted4a_60m_no_0_cox = pd.read_csv('https://www.dropbox.com/s/iewlcsolsni81ui/df_accepted4a_60m_no_0_cox.csv?dl=1')

### Start here to go through preprocessing step-by-step

In [ ]:
print(df_accepted1_sm_inversetransformed.shape) #(678210, 88)

In [ ]:
df_accepted1_sm_inversetransformed['loan_status'].value_counts()

### Add survival 'event' column: loan_status_bool2 (competing risk: 0, 1, 2)

In [ ]:
# 'Prepaid' coded as 1
# 'Default'/'Delinquent'/'Late' coded as 2
# 'Current' coded as 0

def one_two_or_zero(status):  
    if (status == 'Fully Paid') or (status == 'Does not meet the credit policy. Status:Fully Paid'):
        return 1
    elif ((status == 'Charged Off') | (status == 'Late (31-120 days)') | (status == 'In Grace Period')
    | (status == 'Late (16-30 days)') | (status == 'Does not meet the credit policy.  Status:Charged Off') 
    | (status == 'Default')):
        return 2
    else:
        return 0

In [ ]:
df_accepted1_sm_inversetransformed['loan_status_bool2'] = df_accepted1_sm_inversetransformed['loan_status'].apply(lambda x: one_two_or_zero(x))

### Table showing how survival 'event' is coded

In [ ]:
print(df_accepted1_sm_inversetransformed.shape)
df_accepted1_sm_inversetransformed[['loan_status', 'loan_status_bool2']].head() #(678210, 89)

### Add survival 'duration' column: surv1_timedelta

In [ ]:
df_accepted1_sm_inversetransformed['issue_d'] = pd.to_datetime(df_accepted1_sm_inversetransformed['issue_d'])
df_accepted1_sm_inversetransformed['last_pymnt_d'] = pd.to_datetime(df_accepted1_sm_inversetransformed['last_pymnt_d'])
df_accepted1_sm_inversetransformed['next_pymnt_d'] = pd.to_datetime(df_accepted1_sm_inversetransformed['next_pymnt_d'])
df_accepted1_sm_inversetransformed['last_credit_pull_d'] = pd.to_datetime(df_accepted1_sm_inversetransformed['last_credit_pull_d'])

In [ ]:
df_accepted1_sm_inversetransformed['surv1_timedelta'] = df_accepted1_sm_inversetransformed['last_pymnt_d'] - df_accepted1_sm_inversetransformed['issue_d']

In [ ]:
df_accepted1_sm_inversetransformed['surv1_timedelta'] = df_accepted1_sm_inversetransformed['surv1_timedelta'].astype('timedelta64[D]')

In [ ]:
df_accepted1_sm_inversetransformed.shape #(678210, 90) with added surv1_timedelta

### Drop columns

In [ ]:
# Drop columns
df_accepted1_sm_inversetransformed.drop(['revol_bal_joint', 'deferral_term', 'hardship_amount', 'hardship_length',
                                        'hardship_dpd', 'hardship_payoff_balance_amount', 'settlement_amount', 'settlement_percentage',
                                        'settlement_term', 'mths_since_last_delinq', 'next_pymnt_d',
                                        'mths_since_last_major_derog', 'mths_since_rcnt_il',
                                        'total_bal_il', 'il_util', 'open_rv_12m', 'max_bal_bc', 'inq_fi',
                                        'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq',
                                        'hardship_last_payment_amount', 'all_util'], axis=1, inplace=True)
print(df_accepted1_sm_inversetransformed.shape) #678210, 67


### Dropna rows

In [ ]:
df_accepted1_sm_inversetransformed.dropna(subset=['loan_amnt', 'int_rate', 'installment',
 'annual_inc', 'fico_range_low', 'out_prncp', 'total_rec_int', 'term', 'dti'],                           
 inplace=True)
df_accepted1_sm_inversetransformed.shape #(677672, 68)

In [ ]:
df_accepted4_sm_inversetransformed = df_accepted1_sm_inversetransformed.dropna()
print(df_accepted4_sm_inversetransformed.shape) #(505101, 67)
df_accepted4_sm_inversetransformed.isnull().sum().sum() #0

### Read in directly dataframe with columns dropped and NA removed

### Filtered for survival models

In [ ]:
# Filtered for survival models

df_accepted4a_sm_cox = df_accepted4_sm_inversetransformed.filter(['total_pymnt', 'total_rec_prncp', 'out_prncp', 'loan_amnt', 'loan_status_bool2', 
    'loan_status_bool3', 'loan_status', 'term', 'int_rate', 'grade', 'grade_map', 
    'sub_grade_map', 'emp_length', 'home_ownership', 'verification_status',  
    'dti', 'delinq_2yrs', 'fico_range_low', 'inq_last_6mths', 'revol_util', 'out_prncp', 'bc_util', 
    'chargeoff_within_12_mths', 'mths_since_recent_bc', 'mths_since_recent_inq', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'surv1_timedelta'])

df_accepted4a_sm_cox.shape #(505101, 27)

In [ ]:
df_accepted4a_sm_cox['profit_&_loss'] = ((df_accepted4a_sm_cox['total_pymnt'] - df_accepted4a_sm_cox['loan_amnt'])/
                                           df_accepted4a_sm_cox['loan_amnt'])

### Term and Emp_length convert to integers

In [ ]:
df_accepted4a_sm_cox['term'] = df_accepted4a_sm_cox['term'].apply(str)
df_accepted4a_sm_cox['term'] = df_accepted4a_sm_cox.apply(lambda row: "".join(filter(str.isdigit, row['term'])), 
                                  axis=1)
df_accepted4a_sm_cox['term'] = pd.to_numeric(df_accepted4a_sm_cox['term'])

In [ ]:
df_accepted4a_sm_cox['emp_length'] = df_accepted4a_sm_cox['emp_length'].apply(str)
df_accepted4a_sm_cox['emp_length'] = df_accepted4a_sm_cox.apply(lambda row: "".join(filter(str.isdigit, row['emp_length'])), 
                                  axis=1)
df_accepted4a_sm_cox['emp_length'] = pd.to_numeric(df_accepted4a_sm_cox['emp_length'])

In [ ]:
df_accepted4a_sm_cox['GRADE'] = df_accepted4a_sm_cox['grade']
df_accepted4a_sm_cox = df_accepted4a_sm_cox.drop(['grade'], axis=1)

In [ ]:
df_accepted4a_sm_cox['LOAN_AMNT_BINNED1'] = pd.cut(df_accepted4a_sm_cox['loan_amnt'], 
                                                  bins=[0, 1000, 8000, 13950, 20000, 40000],
                                                  labels=['0-1k','1001-8k', '8001-14k',
                                                         '14001-20k', '20001-40k'])


### Bin continuous to categorical

In [ ]:
# Grade vs. Loan Amount
df_accepted4a_sm_cox['BC_UTIL1'] = pd.cut(df_accepted4a_sm_cox['bc_util'], 
                                          bins = [0, 35, 60, 82, 195],
                                          labels = ['0-35%', '35.1-59.1%', '59.2-82.1%', '82.2-194%'])

In [ ]:
# Grade vs. Interest Rate
df_accepted4a_sm_cox['INT_RATE_BINNED1'] = pd.cut(df_accepted4a_sm_cox['int_rate'], 
                                                  bins=[0, 5.5, 10, 12.75, 16, 30],
                                                  labels=['0-5.5%','5.6-10%', '11-12.75%',
                                                         '12.76-16%', '16.01-30%'])


In [ ]:
# Grade vs. FICO
df_accepted4a_sm_cox['FICO_BINNED1'] = pd.cut(df_accepted4a_sm_cox['fico_range_low'], 
                                              bins = [0, 660, 675, 690, 715, 845],
                                              labels = ['0-660', '661-675', '676-690', '691-715', '716-845'])

In [ ]:
# Grade vs. Employment Length
df_accepted4a_sm_cox['EMP_LENGTH_BINNED1'] = pd.cut(df_accepted4a_sm_cox['emp_length'], 
                                          bins = [0, 1, 2, 6, 10],
                                          labels = ['0-1', '1.1-2', '2.2-6', '6.1-10'])

In [ ]:
# Grade vs. Homeownership
df_accepted4a_sm_cox['HOME_OWNERSHIP_OWN_IS_0'] = df_accepted4a_sm_cox['home_ownership'].map({'OWN': 0, 'MORTGAGE': 1, 'RENT': 2}) # own is 0

In [ ]:
# Duration object: convert to months
df_accepted4a_sm_cox['surv1_timedelta_mnth'] = round(df_accepted4a_sm_cox['surv1_timedelta'] / 30.417)

In [ ]:
#Convert from float to int
df_accepted4a_sm_cox['surv1_timedelta_mnth'] = df_accepted4a_sm_cox['surv1_timedelta_mnth'].round(0).astype(int)
print(df_accepted4a_sm_cox.dtypes['surv1_timedelta_mnth'])
df_accepted4a_sm_cox['surv1_timedelta_mnth'].head()

In [ ]:
df_accepted4a_sm_cox.shape #(505101, 35 after add binned categories)

### 36 Month Term

In [ ]:
df_accepted4a_36m_cox = pd.read_csv('https://www.dropbox.com/s/6jom86xuh0bb86h/df_accepted4a_36m_cox.csv?dl=1')
df_accepted4a_36m_cox = df_accepted4a_sm_cox[df_accepted4a_sm_cox['term'] == 36]

### 60 Month Term

In [ ]:
df_accepted4a_60m_cox = pd.read_csv('https://www.dropbox.com/s/mraql0j621xpfl5/df_accepted4a_60m_cox.csv?dl=1')
df_accepted4a_60m_cox = df_accepted4a_sm_cox[df_accepted4a_sm_cox['term'] == 60]

In [ ]:
# subset df without censored observations
df_accepted4a_36m_no_0_cox = df_accepted4a_36m_cox[(df_accepted4a_36m_cox['loan_status_bool2'] == 'Prepaid') | 
                                                   (df_accepted4a_36m_cox['loan_status_bool2'] == 'Default/Delinquent')]

In [ ]:
# subset df without censored observations
df_accepted4a_60m_no_0_cox = df_accepted4a_60m_cox[(df_accepted4a_60m_cox['loan_status'] == 'Prepaid') | 
                                                   (df_accepted4a_60m_cox['loan_status'] == 'Default/Delinquent')]

### READ IN DIRECTLY HERE FILES THAT WILL GO INTO R FILE
### Grade vs. Profit & Loss: 36months 

In [ ]:
#36mnth
df_accepted4a_36m_no_0_cox = pd.read_csv('https://www.dropbox.com/s/m8pnpz5anojf3tb/df_accepted4a_36m_no_0_cox.csv?dl=1')

In [ ]:
df_accepted4a_36m_no_0_cox['GRADE'] = df_accepted4a_36m_no_0_cox['GRADE'].astype('category')

df_accepted4a_36m_no_0_cox['GRADE'].cat.reorder_categories(['A', 'B', 'C', 
                    'D', 'E', 'F', 'G'], inplace=True)

In [ ]:
sns.boxplot(x='GRADE', y='profit_&_loss', data=df_accepted4a_36m_no_0_cox).set_title('Grade vs. Profit & Loss: 36 Months')


In [ ]:
df_accepted4a_36m_cox['GRADE'] = df_accepted4a_36m_cox['GRADE'].astype('category')

df_accepted4a_36m_cox['GRADE'].cat.reorder_categories(['A', 'B', 'C', 
                    'D', 'E', 'F', 'G'], inplace=True)

In [ ]:
df_accepted4a_36m_cox = df_accepted4a_36m_cox['loan_status_bool2'] != 0

In [ ]:
# 60mnth
df_accepted4a_60m_no_0_cox = pd.read_csv('https://www.dropbox.com/s/iewlcsolsni81ui/df_accepted4a_60m_no_0_cox.csv?dl=1')

In [ ]:
df_accepted4a_60m_no_0_cox['GRADE'] = df_accepted4a_60m_no_0_cox['GRADE'].astype('category')

df_accepted4a_60m_no_0_cox['GRADE'].cat.reorder_categories(['A', 'B', 'C', 
                    'D', 'E', 'F', 'G'], inplace=True)

In [ ]:
sns.boxplot(x='GRADE', y='profit_&_loss', data=df_accepted4a_60m_no_0_cox).set_title('Grade vs. Profit & Loss: 60 Months')

### FICO vs. P&L: 36 mnth

In [ ]:
df_accepted4a_36m_no_0_cox['FICO_BINNED1'] = df_accepted4a_36m_no_0_cox['FICO_BINNED1'].astype('category')

df_accepted4a_36m_no_0_cox['FICO_BINNED1'].cat.reorder_categories(
['0-660', '661-675', '676-690', '691-715', '716-845'], inplace=True)

In [ ]:
sns.boxplot(x='FICO_BINNED1', y='profit_&_loss', data=df_accepted4a_36m_no_0_cox).set_title('FICO Score vs. Profit & Loss: 36 Months')

### FICO vs. P&L: 60 mnth

In [ ]:
df_accepted4a_60m_no_0_cox['FICO_BINNED1'] = df_accepted4a_60m_no_0_cox['FICO_BINNED1'].astype('category')

df_accepted4a_60m_no_0_cox['FICO_BINNED1'].cat.reorder_categories(
['0-660', '661-675', '676-690', '691-715', '716-845'], inplace=True)
sns.boxplot(x='FICO_BINNED1', y='profit_&_loss', data=df_accepted4a_60m_no_0_cox).set_title('FICO Score vs. Profit & Loss: 60 Months')

###  Bankcard Utilization vs. P&L

In [ ]:
#36mnth
df_accepted4a_36m_no_0_cox['BC_UTIL1'] = df_accepted4a_36m_no_0_cox['BC_UTIL1'].astype('category')

df_accepted4a_36m_no_0_cox['BC_UTIL1'].cat.reorder_categories(
['0-35%', '35.1-59.1%', '59.2-82.1%', '82.2-194%'], inplace=True)

In [ ]:
sns.boxplot(x='BC_UTIL1', y='profit_&_loss', data=df_accepted4a_36m_no_0_cox).set_title('BC Utilization vs. Profit & Loss: 36 Months')

In [ ]:
#60mnth
df_accepted4a_60m_no_0_cox['BC_UTIL1'] = df_accepted4a_60m_no_0_cox['BC_UTIL1'].astype('category')

df_accepted4a_60m_no_0_cox['BC_UTIL1'].cat.reorder_categories(
['0-35%', '35.1-59.1%', '59.2-82.1%', '82.2-194%'], inplace=True)

In [ ]:
sns.boxplot(x='BC_UTIL1', y='profit_&_loss', data=df_accepted4a_60m_no_0_cox).set_title('BC Utilization vs. Profit & Loss: 60 Months')

### Employment Length vs. P&L

In [ ]:
#36mnth
df_accepted4a_36m_no_0_cox['EMP_LENGTH_BINNED1'] = df_accepted4a_36m_no_0_cox['EMP_LENGTH_BINNED1'].astype('category')

df_accepted4a_36m_no_0_cox['EMP_LENGTH_BINNED1'].cat.reorder_categories(
['0-1', '1.1-2', '2.2-6', '6.1-10'], inplace=True)

In [ ]:
sns.boxplot(x='EMP_LENGTH_BINNED1', y='profit_&_loss', data=df_accepted4a_36m_no_0_cox).set_title('Employment Length vs. Profit & Loss: 36 Months')

In [ ]:
#60mnth
df_accepted4a_60m_no_0_cox['EMP_LENGTH_BINNED1'] = df_accepted4a_60m_no_0_cox['EMP_LENGTH_BINNED1'].astype('category')

df_accepted4a_60m_no_0_cox['EMP_LENGTH_BINNED1'].cat.reorder_categories(
['0-1', '1.1-2', '2.2-6', '6.1-10'], inplace=True)

In [ ]:
sns.boxplot(x='EMP_LENGTH_BINNED1', y='profit_&_loss', data=df_accepted4a_60m_no_0_cox).set_title('Employment Length vs. Profit & Loss: 60 Months')


### Home Ownership vs. P&L

In [ ]:
#36mnth
df_accepted4a_36m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'] = df_accepted4a_36m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'].astype('category')

df_accepted4a_36m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'].cat.reorder_categories(
['Homeowner', 'Mortgage', 'Rent'], inplace=True)

In [ ]:
sns.boxplot(x='HOME_OWNERSHIP_OWN_IS_0', y='profit_&_loss', data=df_accepted4a_36m_no_0_cox).set_title('Home Ownership (Own=0, Mortgage=1, Rent=2) vs. Profit & Loss: 36 Months')


In [ ]:
#60mnth
df_accepted4a_60m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'] = df_accepted4a_60m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'].astype('category')

df_accepted4a_60m_no_0_cox['HOME_OWNERSHIP_OWN_IS_0'].cat.reorder_categories(
['Homeowner', 'Mortgage', 'Rent'], inplace=True)

In [ ]:
sns.boxplot(x='HOME_OWNERSHIP_OWN_IS_0', y='profit_&_loss', data=df_accepted4a_60m_no_0_cox).set_title('Home Ownership (Own=0, Mortgage=1, Rent=2) vs. Profit & Loss: 36 Months')